$$
\newcommand{\mat}[1]{\boldsymbol {#1}}
\newcommand{\mattr}[1]{\boldsymbol {#1}^\top}
\newcommand{\matinv}[1]{\boldsymbol {#1}^{-1}}
\newcommand{\vec}[1]{\boldsymbol {#1}}
\newcommand{\vectr}[1]{\boldsymbol {#1}^\top}
\newcommand{\rvar}[1]{\mathrm {#1}}
\newcommand{\rvec}[1]{\boldsymbol{\mathrm{#1}}}
\newcommand{\diag}{\mathop{\mathrm {diag}}}
\newcommand{\set}[1]{\mathbb {#1}}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\newcommand{\pderiv}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\bb}[1]{\boldsymbol{#1}}
$$

# CS236781: Deep Learning
# Tutorial 5: Optimization

## Introduction

In this tutorial, we will cover:  **TODO**

- Backpropagation
- Optimization
- Automatic differentiation
- PyTorch backward functions
- Bi-level differentiable optimization

In [3]:
# Setup
%matplotlib inline
import os
import sys
import time
import torch
import matplotlib.pyplot as plt

In [4]:
plt.rcParams['font.size'] = 20
data_dir = os.path.expanduser('~/.pytorch-datasets')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Theory Reminders

## Descent-based optimization

As we have seen, training deep neural network is performed iteratively using descent-based optimization.

The general scheme is,

1. Initialize parameters to some $\vec{\Theta}^0 \in \set{R}^P$, and set $k\leftarrow 0$.
2. While not converged:
    1. Choose a direction $\vec{d}^k\in\set{R}^P$
    2. Choose a step size $\eta_k\in\set{R}$
    3. Update: $\vec{\Theta}^{k+1} \leftarrow \vec{\Theta}^k + \eta_k \vec{d}^k$
    4. $k\leftarrow k+1$


Which descent direction to choose?

The one which maximally decreases the loss function $L(\vec{\Theta})$:

$$
\vec{d} =\arg\min_{\vec{d'}} L(\vec{\Theta}+\vec{d'})-L(\vec{\Theta})
\approx
\arg\min_{\vec{d'}}\nabla L(\vec{\Theta})^\top\vec{d'}, \
\mathrm{s.t.} \norm{\vec{d}}_p=1
$$

Choice of norm determines $\vec{d}$. For example,
- $p=1$: Coordinate descent: direction of the largest gradient component.
- $p=2$: Gradient descent: $\vec{d}=-\nabla L(\vec{\Theta})$.

|$p=1$|$p=2$|
|---|---|
|<img src="https://upload.wikimedia.org/wikipedia/commons/e/e3/Coordinate_descent.svg" width="300" /> | <img src="https://upload.wikimedia.org/wikipedia/commons/f/ff/Gradient_descent.svg" width="300" />| 

### Drawbacks and mitigations?

**Susceptible to initialization**

Initializing near local minima can prevent finding better ones.

<center><img src="imgs/sgd-init.png" width="500" /></center>

Can use stochastic gradient to get a different loss surface every iteration.


<center><img src="imgs/sgd-loss.png" width="500" /></center>

**Sensitive to learning rate**

<center><img src="imgs/sgd-lr.png" width="800"/></center>

- Line search (1D minimization):
$$
\eta_k = \arg\min_{\eta'} L(\vec{\Theta}^k+\eta'\vec{d}^k)
$$

- Adaptive LR optimizers, e.g. Adam

- LR scheduling
<center><img src="imgs/sgd-lr-schedule.png" width="500"/></center>

**Zig-zags in narrow "ravines"**

<center><img src="imgs/sgd-zigzag.png" width="300"/></center>

- Momentum: Use previous gradients to build "speed" in the common direction and cancel-out oscillations in opposite directions.

- BatchNorm: Normalizes activations to zero-mean and unit variance (reduces curvature)

- Second-order methods: Use quadratic local approximation of the loss surface, instead of linear.
    - Newton's method: $\vec{d}_k=\mat{H}_k^{-1}\vec{g}_k = \nabla^2 L(\vec{\Theta}_k)^{-1}\nabla L(\vec{\Theta}_k)$.
    - Various other methods which use some estimate of the Hessian.

## The back-propagation algorithm

All the above optimization methods have a crucial thing in common: They require calculation of gradients of the loss w.r.t. to the parameters.

In practical settings when training neural networks we have many different parameters tensors we would like to update separately. Thus, we require the gradient of the loss w.r.t. each of them.

Back-propagation is an efficient way to calculate these gradients using the chain rule.

We represent the application of a model as a **computation graph**.
For example, a simple linear regression model can be represented as:

<center><img src="imgs/backprop-graph.png" width="350"/></center>

Imagine that in this graph we have $N$ variables $\vec{v}^i,\ 1\leq i \leq N$  and functions $f_i$ which compute them from other variables.

The graph is directional, thus assume $\vec{v}^1, \vec{v}^2,\dots,\vec{v}^N$ represents a topological order of the graph (parents before children).

Define also the notation $\delta\vec{v}\triangleq \pderiv{L}{\vec{v}}$.

The forward pass can therefore we written as:

1. For $i=1,2,\dots,N$:
  1. Graph parents of current node: $$\mathcal{P}_i \leftarrow \left\{\vec{v}^j ~\middle\vert~ \vec{v}^j \text{ parent of } \vec{v}^i\right\}$$ 
  2. Function value of current node: $$\vec{v}^i\leftarrow f_i(\mathcal{P}_i)$$

And in the backward pass we traverse the graph in reverse and apply the chain rule:

1. Set $\delta\vec{v}^N=1$.
2. For $i=N,N-1,\dots,1$:
  1. Graph children of current node: $$\mathcal{C}_i \leftarrow \left\{\vec{v}^j ~\middle\vert~ \vec{v}^j \text{ child of } \vec{v}^i\right\}$$  
  2. Chain rule: $$\delta\vec{v}^i\leftarrow \sum_{\vec{v}^j\in\mathcal{C}_i} \delta\vec{v}^j\pderiv{\vec{v}^j}{\vec{v}^i}$$
  
Note that the expression $\delta\vec{v}^j\pderiv{\vec{v}^j}{\vec{v}^i}$ is a "vector"-Jacobian product (VJP).

Backpropagation easily lends itself to a modular and efficient implementation.


Modularity:
- Nodes in the computation graph only need to know how to calculate their own derivatives.
- This is then passed to the parent nodes, which can do the same.


<center><img src="imgs/backprop-modular.png" width="500"/></center>

Efficiency:

- We only need to compute each $\delta\vec{v}^i$ once.
- We in practice never construct the Jacobian, and instead calculate the VJP directly.

Modern automatic-differentiation packages such as PyTorch's `autograd` utilize exactly these tricks to implement backprop in an extremely powerful way.

**Image credits**

Some images in this tutorial were taken and/or adapted from:

- Dr. Roger Grosse, UToronto, cs321
- Fundamentals of Deep Learning, Nikhil Buduma, Oreilly 2017
